In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("video recommendation")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

In [2]:
inputPath="/home/jatin/Documents/spark-notebook-0.6.2-scala-2.11.7-spark-1.6.0-hadoop-2.2.0-with-hive-with-parquet/videoMetaData3csv/videoMetaDataInCsvFormat"

In [6]:
englishStopWords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

In [7]:
#Reading file containing video meta data to make initial RDD
readFile = sc.textFile(inputPath).cache()
readFile.count()

59800

In [8]:
readFile.take(5)

[u'0_04xqwyas,\'Edge of Alaska\': Keeping a Frontier Town Remote,All it takes is one big attraction to change everything. Will McCarthy be next?,"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska"',
 u'0_0m8u8g4c,\'MythBusters\': Shark Feeding Frenzy Defense,"While testing the repelling abilities of ""essence of dead shark"", Jamie hopes his beret will serve as more than just a fashion statement.","reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, shark myths, feeding frenzy, mythbusters shark tests, shark armor"',
 u'0_0pd5gia9,\'MythBusters\': A MythBusting Lightsaber Duel,Jamie and Adam put the myth of higher ground to the test in this epic lightsaber battle.,"reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, lightsaber dueling"',
 u'0_0uwz3tkl,Behind the Scenes - Lucky Influencers: What Was Your Fashion First?,"5 fashion industry influencers, Va$shtie Kola, Lily Kwong, Amirah Kassem,

In [16]:
# Converting string into (ID,remainingString) format
def formatInput(item):
     name = item.split(",")[0]
     return (name,item.replace(name+",",""))

nameFromCsv = readFile.map(formatInput)
#                                name = item.split(",")(0)
#                               (name,item.replace(name+",",""))
#                               }

In [28]:
nameFromCsv.take(2)

[(u'0_04xqwyas',
  u'\'Edge of Alaska\': Keeping a Frontier Town Remote,All it takes is one big attraction to change everything. Will McCarthy be next?,"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska"'),
 (u'0_0m8u8g4c',
  u'\'MythBusters\': Shark Feeding Frenzy Defense,"While testing the repelling abilities of ""essence of dead shark"", Jamie hopes his beret will serve as more than just a fashion statement.","reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, shark myths, feeding frenzy, mythbusters shark tests, shark armor"')]

In [30]:
reserveWords = sc.broadcast(englishStopWords)

In [79]:
import re
def bagOfWords(item):
    regexSpecialChars = """[^a-zA-Z0-9\s]"""
    noSymbolsLine = re.sub(regexSpecialChars, " ",item)
    stopWords = reserveWords.value
    print type(stopWords)
    listOfWords = noSymbolsLine.split()
    nonEmptyWords = filter(lambda x :x != "",listOfWords)
    removeReserveWords = []
    for word in nonEmptyWords:
        if word not in stopWords:
            removeReserveWords.append(word.lower())
    return removeReserveWords

In [80]:
bagOfWords("'Edge of Alaska': Keeping a Frontier 1232 Town  @ #Remote,All it takes is () *%* one big attraction to change everything. Will McCarthy be next?,\"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska\"")

<type 'list'>


['edge',
 'alaska',
 'keeping',
 'frontier',
 '1232',
 'town',
 'remote',
 'all',
 'takes',
 'big',
 'attraction',
 'change',
 'will',
 'mccarthy',
 'reality',
 'tv',
 'series',
 'edge',
 'alaska',
 'mccarthy',
 'alaska',
 'frontier',
 'town',
 'grid',
 'living',
 'takeetna',
 'alaska']

In [74]:
cleandData = nameFromCsv.mapValues(bagOfWords)

In [77]:
# Cross Checking count of data after cleaning 
numDocs = cleandData.count()

In [78]:
print numDocs

59800


In [ ]:
readFile.unpersist()
# Finding term frequency for each term in video metaData (term,freq)
termDocumentFrequencies = cleandData.mapValues{inp =>
  inp.groupBy(x => x).mapValues(_.size)
#Grouping based on uniqueness and computing size of each group to obtain (term,count) pairs
}   